In [1]:
import sys
from pathlib import Path


# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import *
from spytial.annotations import *

In [2]:
# Setup for performance metrics
import random
from time import sleep
perf_base = "spytial_perf"
def get_perf_path(structure, size):
    return perf_base + "_" + structure + "_" + f"{size}.json"
PI = 30
SIZES = [5, 10, 25, 50]

# Memoization Data Structure TODO
m and s tables computed by Matrix-Chain-Order

In [3]:
# Memoization Data Structure
from math import inf
from functools import lru_cache

def memoized_matrix_chain(p):
    """
    p: dims vector of length n+1 (A1 is p[0]x p[1], ..., An is p[n-1]x p[n])
    returns: m, s where m[i][j] = min cost, s[i][j] = optimal split k
    1-indexed to match CLRS.
    """
    n = len(p) - 1
    m = [[None]*(n+1) for _ in range(n+1)]
    s = [[None]*(n+1) for _ in range(n+1)]

    @lru_cache(None)
    def lookup(i, j):
        if i == j:
            m[i][j] = 0
            return 0
        best, best_k = inf, None
        for k in range(i, j):
            left = lookup(i, k)
            right = lookup(k+1, j)
            cost = left + right + p[i-1]*p[k]*p[j]
            if cost < best:
                best, best_k = cost, k
        m[i][j] = best
        s[i][j] = best_k
        return best

    lookup(1, n)
    return m, s



##CLRS (Cormen, Leiserson, Rivest, Stein - Introduction to Algorithms, Chapter 15) uses the bottom-up approach. They describe the algorithm with nested loops that fill the tables iteratively, exactly like your bottom_up_matrix_chain function. This is the standard presentation in the textbook, as it aligns with how they explain dynamic programming tables.
def bottom_up_matrix_chain(p):
    n = len(p)-1
    m = [[0 if i==j else inf for j in range(n+1)] for i in range(n+1)]
    s = [[None]*(n+1) for _ in range(n+1)]
    for L in range(2, n+1):             # chain length
        for i in range(1, n-L+2):
            j = i+L-1
            m[i][j] = inf
            for k in range(i, j):
                q = m[i][k] + m[k+1][j] + p[i-1]*p[k]*p[j]
                if q < m[i][j]:
                    m[i][j] = q
                    s[i][j] = k
    return m, s

def print_tables(m, s):
    # compact CLRS-style triangular display
    n = len(m)-1
    def row_fmt(row): return " ".join(("{:>6}".format("" if x is None or x==float('inf') else x) for x in row[1:]))
    print("m-table:")
    for i in range(1, n+1):
        row = [None]*(i) + m[i][i:]  # leading blanks
        print(row_fmt(row))
    print("\ns-table (k splits):")
    for i in range(1, n+1):
        row = [None]*(i) + s[i][i:]
        print(row_fmt(row))

def parenthesize(s, i, j):
    if i == j: return f"A{i}"
    k = s[i][j]
    return f"({parenthesize(s, i, k)}{parenthesize(s, k+1, j)})"


![memo](img/memoization.png)

In [6]:
## ACTUALLY, I'm just not sure?


# Example (CLRS Fig. 15.5 uses p = [30,35,15,5,10,20,25])
p = [30,35,15,5,10,20,25]
m, s = bottom_up_matrix_chain(p)   # or memoized_matrix_chain(p)
m = hideAtom(selector="{ f : float | @:f = inf}")(m)

# Build up selectors incrementally
ROW_COL_ELEM = "((list).(idx & (list->int->list)).(idx & list->int->int))"

SAME_COL_DIFF_ROW = f"{{ v1, v2 : int | (some r1, r2, c : int | r1 != r2 and (r1->c->v1 + r2->c->v2) in {ROW_COL_ELEM} ) }}"
print(SAME_COL_DIFF_ROW)

# Now use the projections to define same row/col
SAME_ROW = f"{{v1, v2 : int | v1 != v2 and some r : int | (r->v1 + r->v2) in {ROW_ELEM}}}"
SAME_COL = f"{{v1, v2 : int | v1 != v2 and some c : int | (c->v1 + c->v2) in {COL_ELEM}}}"

print(SAME_ROW)


# Apply layout
m = orientation(selector=SAME_ROW, directions=["directlyRight"])(m)
m = orientation(selector=SAME_COL, directions=["directlyBelow"])(m)
m = align(selector=SAME_ROW, direction="horizontal")(m)
m = align(selector=SAME_COL, direction="vertical")(m)


# ROW_COL_ELEM = "((list).(idx & (list->int->list)).(idx & list->int->int))"

# # ROW_ELEM: pairs of (row_idx, value) - project away the column
# # For each (r, c, v) in ROW_COL_ELEM, we get (r, v)
# ROW_ELEM = f"{{r, v : int | some c : int | (r->c->v) in {ROW_COL_ELEM}}}"

# # COL_ELEM: pairs of (col_idx, value) - project away the row  
# # For each (r, c, v) in ROW_COL_ELEM, we get (c, v)
# COL_ELEM = f"{{c, v : int | some r : int | (r->c->v) in {ROW_COL_ELEM}}}"


# SAME_ROW=f"{{s1, s2 : int | (some r, c1, c2 : int | (c1->s1 + c2->s2) in r.({ROW_COL_ELEM}) ) }}"

# print(SAME_ROW)

# def into_matrix(xxs):
    
#     xxs = orientation(selector=f"{{s1, s2 : object | (some r, c1, c2 : int | (r->c1->s1 + r->c2->s2) in {ROW_COL_ELEM} ) }}", directions=["directlyLeft"])(xxs)
#     return xxs

# m = into_matrix(m)

evaluate(m, method="browser")
#diagram(m)
#diagram(s)

{ v1, v2 : int | (some r1, r2, c : int | r1 != r2 and (r1->c->v1 + r2->c->v2) in ((list).(idx & (list->int->list)).(idx & list->int->int)) ) }
{v1, v2 : int | v1 != v2 and some r : int | (r->v1 + r->v2) in {r, v : int | some c : int | (r->c->v) in ((list).(idx & (list->int->list)).(idx & list->int->int))}}


'/var/folders/80/rtptthbx3zq0tb06wwzmck_40000gq/T/tmpej4nbrgb.html'

## Performance

In [ ]:
STRUCTURE = "memoization"
for size in SIZES:
    # Generate random dimension vector (size+1 dimensions for size matrices)
    p = [random.randint(5, 50) for _ in range(size + 1)]
    
    m, s = bottom_up_matrix_chain(p)
    
    # Apply visualization transformations
    m_vis = hideAtom(selector="{ f : float | @:f = inf}")(m)
    m_vis = into_matrix(m_vis)
    
    print(f"{STRUCTURE}({size} matrices): Rendering with perf_iterations={PI}...")
    evaluate(m_vis, method="browser", perf_path=get_perf_path(STRUCTURE, size), perf_iterations=PI)
    sleep(2)

print("\n✅ Performance data collection complete!")
print(f"📊 Check {perf_base}_*.json files for metrics")